In [1]:
from os import listdir
from os.path import isfile, join
import json
import gensim
LabeledSentence = gensim.models.doc2vec.LabeledSentence
import multiprocessing

ModuleNotFoundError: No module named 'gensim'

In [ ]:
docList = []
docList = [f for f in listdir("../dataset/papers") if f.endswith('.txt')]

In [ ]:
docIndex = []
for d in docList:
    docIndex.append(int(d.split('.')[0].partition("file")[-1]))
docIndex.sort()

In [ ]:
with open("../arxivdataset/arxivData.json", "r") as read_file:
    papers = json.load(read_file)

In [ ]:
docLabels = []
docLabels = [papers[i - 1]['title'] for i in docIndex]

In [ ]:
data = []
for doc in docList:
    data.append(open("../dataset/papers/" + doc, 'r', encoding = "utf-8").read())

In [ ]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
       self.labels_list = labels_list
       self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield LabeledSentence(words=doc.split(), tags=[self.labels_list[idx]])

In [ ]:
it = LabeledLineSentence(data, docLabels)

cores = multiprocessing.cpu_count()

model = gensim.models.Doc2Vec(dm=1, dm_mean=1, vector_size=200, window=8, min_count=10, epochs=10, workers=cores)

model.build_vocab(it)
model.train(it, total_examples=model.corpus_count, epochs=model.iter)

model.save("arxiv.model")
